In [4]:
#  Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
"""Convolutional Neural Network Estimator for MNIST, built with tf.layers."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib

tf.logging.set_verbosity(tf.logging.INFO)


def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # MNIST images are 28x28 pixels, and have one color channel
  input_layer = tf.reshape(features, [-1, 28, 28, 1])

  # Convolutional Layer #1
  # Computes 32 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2
  # Computes 64 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 14, 14, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 14, 14, 64]
  # Output Tensor Shape: [batch_size, 7, 7, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 64]
  # Output Tensor Shape: [batch_size, 7 * 7 * 64]
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 7 * 7 * 64]
  # Output Tensor Shape: [batch_size, 1024]
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == learn.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout, units=10)

  loss = None
  train_op = None

  # Calculate Loss (for both TRAIN and EVAL modes)
  if mode != learn.ModeKeys.INFER:
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == learn.ModeKeys.TRAIN:
    train_op = tf.contrib.layers.optimize_loss(
        loss=loss,
        global_step=tf.contrib.framework.get_global_step(),
        learning_rate=0.001,
        optimizer="SGD")

  # Generate Predictions
  predictions = {
      "classes": tf.argmax(
          input=logits, axis=1),
      "probabilities": tf.nn.softmax(
          logits, name="softmax_tensor")
  }

  # Return a ModelFnOps object
  return model_fn_lib.ModelFnOps(
      mode=mode, predictions=predictions, loss=loss, train_op=train_op)


def main(unused_argv):
  # Load training and eval data
  mnist = learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = learn.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

  # Set up logging for predictions
  # Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

  # Train the model
  mnist_classifier.fit(
      x=train_data,
      y=train_labels,
      batch_size=100,
      steps=20000,
      monitors=[logging_hook])

  # Configure the accuracy metric for evaluation
  metrics = {
      "accuracy":
          learn.MetricSpec(
              metric_fn=tf.metrics.accuracy, prediction_key="classes"),
  }

  # Evaluate the model and print results
  eval_results = mnist_classifier.evaluate(
      x=eval_data, y=eval_labels, metrics=metrics)
  print(eval_results)


if __name__ == "__main__":
  tf.app.run()

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x122f3ecf8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': None}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator w

/usr/local/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:248: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-3
INFO:tensorflow:Saving checkpoints for 4 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:loss = 2.29924, step = 4
INFO:tensorflow:probabilities = [[ 0.09209193  0.11317121  0.09243919  0.09827472  0.08914029  0.10215061
   0.06999686  0.10268234  0.10341743  0.13663548]
 [ 0.10900951  0.10509806  0.091031    0.10334567  0.08276258  0.09514423
   0.09102036  0.09817632  0.10520419  0.11920807]
 [ 0.10584006  0.09932572  0.08799886  0.09543517  0.09664582  0.09496834
   0.08980121  0.11090978  0.09027678  0.12879829]
 [ 0.10539455  0.09677635  0.09681539  0.09525227  0.08163457  0.10659177
   0.08639016  0.105566    0.08948459  0.13609444]
 [ 0.10260332  0.1071052   0.08946811  0.09852364  0.09285259  0.1050717
   0.0886664   0.10417857  0.10077075  0.11075968]
 [ 0.09798287  0.10343583  0.08791278  0.10492631  0.08519239  0.10370129
   0.0865135   0.10268

INFO:tensorflow:probabilities = [[ 0.10261147  0.10300399  0.09180744  0.09208079  0.0892873   0.09187005
   0.08686052  0.11907927  0.09386797  0.12953126]
 [ 0.10354863  0.10584715  0.10068198  0.08898626  0.08482821  0.09688253
   0.08984088  0.10406885  0.10044917  0.12486639]
 [ 0.09836093  0.09977661  0.09944364  0.09632629  0.09471466  0.09927604
   0.08466735  0.09191161  0.10802118  0.12750165]
 [ 0.09903106  0.09078666  0.08141159  0.11541874  0.09305955  0.09182369
   0.07600421  0.10944335  0.09615285  0.14686838]
 [ 0.11317343  0.1021046   0.10021404  0.09261936  0.08657177  0.09690031
   0.08253657  0.09711765  0.0926505   0.13611178]
 [ 0.10642864  0.09638187  0.10017771  0.09379852  0.08428888  0.09873051
   0.0780331   0.10525542  0.11590107  0.12100431]
 [ 0.10506672  0.10455935  0.08635601  0.1028067   0.09245095  0.09741866
   0.07691897  0.11046029  0.09723005  0.12673222]
 [ 0.11879773  0.10048245  0.09241919  0.09010145  0.0852212   0.08730689
   0.08770122  0.12

INFO:tensorflow:global_step/sec: 2.647
INFO:tensorflow:loss = 2.29749, step = 104 (37.780 sec)
INFO:tensorflow:probabilities = [[ 0.11857671  0.11593056  0.10165862  0.09216258  0.08260427  0.09192222
   0.09203777  0.10468376  0.09790998  0.10251354]
 [ 0.11302445  0.08858893  0.09318367  0.10250945  0.0948088   0.09473406
   0.0934338   0.1063611   0.09774502  0.11561067]
 [ 0.09450883  0.09820622  0.09661757  0.1112859   0.09356201  0.10070848
   0.08593224  0.1001725   0.10243122  0.11657498]
 [ 0.10464489  0.09170181  0.08209858  0.09703609  0.08790541  0.1088109
   0.09130299  0.10596066  0.09606433  0.1344744 ]
 [ 0.10432532  0.09697132  0.09512155  0.09594557  0.08773436  0.09563199
   0.08305861  0.09931853  0.10908939  0.13280335]
 [ 0.10102098  0.10537424  0.09799726  0.10029767  0.09201064  0.08737613
   0.09937612  0.09916157  0.09879819  0.1185872 ]
 [ 0.10377435  0.10245034  0.09051837  0.09487245  0.08869486  0.10178164
   0.08845896  0.10093871  0.10320946  0.12530082]

...

INFO:tensorflow:Saving checkpoints for 3432 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:probabilities = [[ 0.00023269  0.9545027   0.00652538  0.00808109  0.00293562  0.0023176
   0.00113706  0.00833637  0.00644535  0.0094861 ]
 [ 0.00365252  0.00002758  0.00001474  0.0003041   0.18065603  0.60981476
   0.00149731  0.00741526  0.02920452  0.16741322]
 [ 0.00616628  0.00072415  0.00096434  0.02062315  0.11467506  0.01150011
   0.03194837  0.02901434  0.01067689  0.77370721]
 [ 0.01503148  0.01442535  0.62575746  0.1216631   0.00076128  0.01341638
   0.01801913  0.00084066  0.18824555  0.00183965]
 [ 0.97293854  0.00000008  0.00821904  0.00693413  0.00000142  0.00834784
   0.00018707  0.00004049  0.00324157  0.00008979]
 [ 0.00126876  0.04981161  0.00721873  0.00412199  0.56435579  0.01364452
   0.0171458   0.08542261  0.07108101  0.18592927]
 [ 0.0003204   0.00292099  0.00168261  0.0056533   0.00020361  0.0175278
   0.00004058  0.00466146  0.96568489  0.00130445]
 [ 0.0001

INFO:tensorflow:global_step/sec: 2.90364
INFO:tensorflow:loss = 0.438403, step = 3504 (34.439 sec)
INFO:tensorflow:probabilities = [[ 0.00047568  0.00002541  0.00805017  0.00049865  0.00119629  0.00034936
   0.98231202  0.00000902  0.00562767  0.00145563]
 [ 0.00226616  0.00342051  0.00021664  0.00052085  0.0060319   0.92000932
   0.00093319  0.00484756  0.04370484  0.01804909]
 [ 0.00190151  0.30197272  0.0477175   0.05256892  0.00365905  0.00241005
   0.00160291  0.26766014  0.16069014  0.159817  ]
 [ 0.00198112  0.00015477  0.01057888  0.85543776  0.00017749  0.04180161
   0.00342169  0.02360915  0.05610368  0.00673399]
 [ 0.99328411  0.          0.00028538  0.00003511  0.00000017  0.0061614
   0.00016843  0.00000179  0.00006161  0.00000201]
 [ 0.00001565  0.0002838   0.0001586   0.00988357  0.17971563  0.00405827
   0.00013281  0.0166345   0.02003416  0.76908302]
 [ 0.00384031  0.00401065  0.00139316  0.03282497  0.07562026  0.01674909
   0.0055092   0.10594835  0.11415988  0.63994

INFO:tensorflow:probabilities = [[ 0.0000474   0.00000061  0.00000036  0.00044859  0.00017957  0.00019841
   0.00000007  0.99311197  0.00005269  0.00596035]
 [ 0.00055887  0.00000126  0.99447906  0.00068754  0.00002233  0.00031084
   0.0032996   0.00002522  0.00061074  0.00000445]
 [ 0.00000192  0.00002305  0.99640381  0.00178662  0.0000039   0.0000047
   0.00049863  0.00000124  0.00127187  0.00000436]
 [ 0.00007324  0.00000959  0.00000931  0.00000982  0.90018588  0.00820014
   0.00047602  0.006363    0.00150055  0.08317236]
 [ 0.09136557  0.00000365  0.75972825  0.04943107  0.01598019  0.00738141
   0.06929385  0.0000281   0.00623474  0.00055314]
 [ 0.00032865  0.00000023  0.98068672  0.01860975  0.00000083  0.0000944
   0.00018437  0.00000272  0.00009148  0.00000093]
 [ 0.98277283  0.00000104  0.00019143  0.00301968  0.00000412  0.01090277
   0.00006679  0.0005935   0.00200233  0.00044548]
 [ 0.94321847  0.00001246  0.03522852  0.00231284  0.00005887  0.00711832
   0.00668723  0.0000

INFO:tensorflow:global_step/sec: 2.95901
INFO:tensorflow:loss = 0.486033, step = 3604 (33.795 sec)
INFO:tensorflow:probabilities = [[ 0.00206933  0.00002223  0.01307843  0.98067492  0.00000026  0.00320251
   0.00001459  0.00010636  0.00079984  0.00003151]
 [ 0.00376482  0.01593347  0.00046174  0.01455964  0.054131    0.07893467
   0.00649332  0.03811023  0.02879587  0.75881523]
 [ 0.00042417  0.95478505  0.00522406  0.00869729  0.00038522  0.00707444
   0.00805407  0.00415612  0.00972199  0.00147757]
 [ 0.00107353  0.00125336  0.00802603  0.03966108  0.01640381  0.05229358
   0.01870069  0.00066593  0.83288223  0.02903982]
 [ 0.00003542  0.00000241  0.9895649   0.01021601  0.00000001  0.00004103
   0.0000532   0.00000085  0.00008622  0.00000001]
 [ 0.02445355  0.00046188  0.00175088  0.02415468  0.00315527  0.05024967
   0.00132125  0.1760442   0.0246677   0.69374096]
 [ 0.00013803  0.00125614  0.81147176  0.00497948  0.00006404  0.00197873
   0.0303975   0.00002017  0.14962453  0.0000

KeyboardInterrupt: 